### Momentum-Based Strategy
#### Executive Summary
##### Objective
This project explores a momentum-based long-short strategy using the monthly industry return dataset. Momentum investing operates on a simple rule that assets which have done well recently will continue to do in the near future, while poor performers will keep underperforming. Our approach systematically ranks industries into deciles based on their past returns and constructs a portfolio that aims to profit from this trend.

- Formation Period: We compute cumulative returns over the past 12 months (excluding the most recent month) to rank the chosen industries
- Ranking & Grouping: Industries are sorted into deciles. We go long on the top 3 deciles (“winners”) and short the bottom decile
- Portfolio Construction:
    - The winners are equally weighted with a total long allocation of 130%
    - The losers are equally weighted with a total short allocation of -30%
- The rebalancing was done monthly, and we tracked how the combined portfolio performed over time.

This creates a dollar-neutral strategy with more emphasis on long positions, which helps manage volatility while exploiting momentum signals.

##### Why This Strategy Is Competitive
While the full-sample Sharpe ratio of ~0.74 may appear moderate, it does not tell the full story. Figure 1 shows that this strategy steadily compounds wealth over the long run, and if you had invested 1,000 dollars at the beginning of the period, it would’ve grown into over $110 million by the end. Below are the reasons why this strategy remains a strong competitor in both absolute and risk-adjusted terms:

- Fully Invested, Always Active: Unlike breakout strategies that may hold cash when no signal is present, our momentum strategy is always allocated — either long or short. This ensures consistent exposure to return-generating opportunities.
- Balanced Weights: By allocating 130% to longs and 30% to shorts, we capitalize more on winners while limiting risk from the shorts.
- Diversified Returns: Since returns are diversified across many different industries, the strategy benefits from the diversification effect, meaning that it reduces the risk that one bad pick will ruin the portfolio, creating more stable momentum signals.
- Cumulative: Even though the loser portfolio drags performance during crises (e.g., dot-com, 2008 financial crisis), the compounding from long positions more than compensates, creating strong cumulative growth.

##### Performance Summary
- Total Return: As shown in Figure 1, a 1000 dollar investment in this strategy grows to over \$110 million. This explosive compounding is what makes the strategy shine from a total-return perspective.
- CAGR: At 18.43%, the compounded growth rate is strong, especially given the long historical sample from 1929 to 2023.
- Max Drawdown: The worst observed drawdown of -73.4% reflects real-world periods of market stress (e.g., 2008). While high, it’s consistent with trend-following strategies that remain fully invested.
- Sortino Ratio: Our Sortino ratio of 1.02 outperforms our Sharpe ratio, showing that downside risk was better managed than total volatility would suggest. This is a strong indicator that returns came with relatively fewer severe negative months.
- Risk-Adjusted Return: As seen in Figure 3, our 5-year rolling Sharpe ratio peaked at 2.05 in April 1946, suggesting that if the sampling period is more favorable, our strategy could rank near the top in risk-adjusted returns too.

##### Conclusion
Our strategy blends robust long-term performance with disciplined risk exposure. It doesn’t just chase returns — it systematically applies momentum filters, rebalances consistently, and uses leverage smartly. While the Sharpe ratio may seem average over the entire period, our total return and CAGR metrics suggest otherwise.

More importantly, the strategy's performance improves significantly when viewed through shorter, more favorable windows, where risk-adjusted metrics like Sharpe and Sortino ratios reflect efficiency. This makes our strategy a well-rounded contender that delivers across multiple evaluation criteria.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
%precision 4
pd.options.display.float_format = '{:.4f}'.format
%config InlineBackend.figure_format = 'retina'

In [2]:
ind49 = (
    pd.read_csv(
        filepath_or_buffer='data_ind49.csv',
        parse_dates=['Date'],
        na_values=[-99.99, -999]
    )
    .assign(Date=lambda x: x['Date'].dt.to_period('M'))
    .set_index('Date')
    .sort_index()
    .rename_axis(columns=['Industry'])
    .div(100)
)
ff3 = (
    pd.read_csv(
        filepath_or_buffer='data_ff3.csv',
        parse_dates=['Date'],
        na_values=[-99.99, -999]
    )
    .assign(Date=lambda x: x['Date'].dt.to_period('M'))
    .set_index('Date')
    .rename_axis(columns=['Factor'])
    .div(100)
)

FileNotFoundError: [Errno 2] No such file or directory: 'data_ind49.csv'

In [ ]:
mom = (
    pd.read_csv(
        filepath_or_buffer='data_mom.csv',
        parse_dates=['Date'],
        na_values=[-99.99, -999]
    )
    .assign(Date=lambda x: x['Date'].dt.to_period('M'))
    .set_index('Date')
    .rename_axis(columns=['Factor'])
    .div(100)
)

ff5 = (
    pd.read_csv(
        filepath_or_buffer='data_ff5.csv',
        parse_dates=['Date'],
        na_values=[-99.99, -999]
    )
    .assign(Date=lambda x: x['Date'].dt.to_period('M'))
    .set_index('Date')
    .rename_axis(columns=['Factor'])
    .div(100)
)

In [ ]:
cleaned_data = ind49.dropna(axis=1, how='any')
past_11 = cleaned_data.add(1).rolling(12).apply(np.prod) - 1

In [ ]:
def momentum(formation):
    end_measurement = (formation - 1).asfreq('M')
    ret_12 = past_11.loc[end_measurement]
    ret_12 = ret_12.reset_index()
    ret_12['quintile'] = pd.qcut(ret_12.iloc[:, 1], 10, labels=False)
    winners = ret_12[ret_12.quintile >= 8]
    losers = ret_12[ret_12.quintile <= 1]
    winners_rets = cleaned_data.loc[(formation + 1).asfreq('M'), cleaned_data.columns.isin(winners['Industry'])]
    losers_rets = cleaned_data.loc[(formation + 1).asfreq('M'),cleaned_data.columns.isin(losers['Industry'])]
    w_long = 1.3 / len(winners_rets)      
    w_short = -0.3 / len(losers_rets)  
    WinnersRets = (winners_rets * w_long).sum()
    LosersRets = (losers_rets * w_short).sum()
    MomentumProfit = (winners_rets * w_long).sum() + (losers_rets * w_short).sum()
    return MomentumProfit, WinnersRets, LosersRets

formation_dates = cleaned_data.index[12:-1]
returns = []
dates = []
for date in cleaned_data.index[12:-1]:
    mom_ret, win_ret, los_ret = momentum(date)
    returns.append([mom_ret, win_ret, los_ret])
    dates.append(date)
momentumData = pd.DataFrame(returns, columns=['Momentum Returns', 'Winners', 'Losers'], index=dates)

In [ ]:
#| label: fig-cumulative-returns
#| fig-cap: "Cumulative Returns of the Momentum Strategy"
momentumData.add(1).cumprod().mul(1000).plot(
    figsize=(10, 6),
    title='Cumulative Returns (Momentum Strategy)',
    linewidth=2
)
plt.yscale('log')
plt.ylabel('Value ($)')
plt.xlabel('Date')
plt.legend(title='Portfolio')
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
plt.tight_layout()
plt.show()

As shown in @fig-cumulative-returns, This chart shows the value of $1,000 invested in winners, losers, and our combined momentum strategy from the early 1930s onward. The blue line represents the momentum strategy, orange line represents the winners and the green line represents the losers portfolio.

In [ ]:
def Volatility(x, ann_fac=np.sqrt(12)):
    return ann_fac * x.std()
def make_sharpe(rf_series):
    def Sharpe(x):
        aligned_rf = rf_series.reindex(x.index)
        excess = x - aligned_rf
        return np.sqrt(12) * excess.mean() / excess.std()
    return Sharpe
def make_sortino(rf_series):
    def Sortino(x):
        aligned_rf = rf_series.reindex(x.index)
        excess = x - aligned_rf
        downside = excess[excess < 0].std()
        return np.sqrt(12) * excess.mean() / downside if downside else np.nan
    return Sortino
def Total_Return(x):
    return (1 + x).prod() - 1
def CAGR(x, ann_fac=12):
    return (1 + x).prod() ** (ann_fac / x.count()) - 1
def Drawdown(x):
    price = (1 + x).cumprod()
    return (price / price.cummax()).min() - 1

In [ ]:
strategy_stats = (
    momentumData[['Momentum Returns']]
    .agg([Total_Return, CAGR, Volatility,
          make_sharpe(ff3['RF']),
          make_sortino(ff3['RF']), 
          Drawdown])
    .rename_axis(index='Statistic')
    .round(4)
    .transpose()
)

strategy_stats['Strategy'] = 'Momentum'
strategy_stats = strategy_stats.reset_index(drop=True)
strategy_stats

In [ ]:
#| label: fig-performance-stats
#| fig-cap: "Performance Metrics Including Sharpe, Sortino, Volatility, CAGR, and Drawdown"
filtered_stats = strategy_stats.drop(columns=['Total_Return', 'CAGR', 'Drawdown', 'Volatility'])
long_stats_filtered = filtered_stats.melt(id_vars='Strategy', var_name='Statistic', value_name='Value')
side_stats = strategy_stats.drop(columns=['Total_Return', 'Sharpe', 'Sortino'])
long_side_stats = side_stats.melt(id_vars='Strategy', var_name='Statistic', value_name='Value')

fig, ax = plt.subplots(1, 2, figsize=(5, 5), sharey=True)
sns.barplot(data=long_stats_filtered, x='Statistic', y='Value', hue='Strategy', palette='hls', ax=ax[0])
ax[0].set_title('Sharpe & Sortino')
ax[0].set_xlabel('Statistic')
ax[0].set_ylabel('Value (decimals)')
ax[0].tick_params(axis='x', rotation=45)

sns.barplot(data=long_side_stats, x='Statistic', y='Value', hue='Strategy', ax=ax[1])
ax[1].set_title('Volatility & Drawdown')
ax[1].set_xlabel('Metrics')
ax[1].set_ylabel('')
ax[1].tick_params(axis='x', rotation=45)
for axis in ax:
    axis.set_ylim(-0.8, 1.1)

fig.suptitle('Performance Statistics for Momentum Strategy', fontsize=14)
ax[0].get_legend().remove()
ax[1].get_legend().remove()
plt.tight_layout()
plt.subplots_adjust(top=0.85) 
plt.show()

@fig-performance-stats summarizes the key risk and return metrics for our strategy. The Sortino ratio stands out as stronger than the Sharpe, highlighting lower downside volatility.

In [ ]:
#| label: fig-rolling-sharpe
#| fig-cap: "5-Year Rolling Sharpe Ratio of the Momentum Strategy"
momentum_rf_adj = momentumData['Momentum Returns'].sub(ff3['RF'].reindex(momentumData.index))
rolling_sharpe = (
    momentum_rf_adj
    .rolling(window=60)
    .apply(lambda x: np.sqrt(12) * x.mean() / x.std())
)

rolling_sharpe.index = rolling_sharpe.index.to_timestamp()
max_sharpe = rolling_sharpe.max()
max_date = rolling_sharpe.idxmax()
ax = rolling_sharpe.plot(title='Rolling 5-Year Sharpe Ratio (Momentum Strategy)', figsize=(8, 4))
ax.axhline(y=2, color='gray', linestyle='--', alpha=0.5)
ax.scatter(max_date, max_sharpe, color='red', zorder=5)
ax.text(max_date, max_sharpe + 0.01, f'Max: {max_sharpe:.2f}', color='red', ha='center')
ax.text(max_date, max_sharpe - 0.15, f'@ {max_date.strftime("%Y-%m")}', color='red', ha='center')
ax.set_ylabel('Sharpe Ratio')
ax.set_xlabel('Date')
plt.tight_layout()
plt.show()

@fig-rolling-sharpe shows that the strategy’s Sharpe ratio peaked at 2.05 in April 1946. This demonstrates that, depending on the sample window, our strategy could have ranked much higher on risk-adjusted return.

In [ ]:
def write_csv(portfolio_returns, group_number):
    """
    Write portfolio returns to a CSV file named Group_XX.csv with Date as YYYY-MM.
    Parameters:
    - portfolio_returns: pandas Series or DataFrame with monthly returns (as decimals) and Date index
    - group_number: int, your two-digit group number (e.g., 1 for Group_01)
    Output:
    - Saves Group_XX.csv with two columns: Date (YYYY-MM) and Group_XX (returns as decimals)
    """
    # Make two-digit group number
    group_str = f"{group_number:02d}"
    filename = f"Group_{group_str}.csv"

    # Convert returns to DataFrame if Series
    if isinstance(portfolio_returns, pd.Series):
        df = portfolio_returns.to_frame(name=f"Group_{group_str}")
    else:
        df = portfolio_returns.rename(columns={portfolio_returns.columns[0]: f"Group_{group_str}"})

    # Convert index to PeriodIndex with monthly frequency and format as YYYY-MM
    if isinstance(df.index, pd.PeriodIndex):
        if df.index.freq != 'ME':
            df.index = df.index.to_timestamp().to_period('M')
    elif isinstance(df.index, pd.DatetimeIndex):
        df.index = df.index.to_period('M')
    else:
        raise ValueError("Index must be a datetime or PeriodIndex")

    # Rename index to Date
    df.index.name = "Date"

    # Write to CSV
    df.to_csv(filename, index=True)
    print(f"Saved portfolio returns to {filename}")

# # Example usage:
# dates = pd.date_range(start="1931-07-01", end="1936-06-01", freq="MS")
# returns = pd.Series([0.0025, 0.01, -0.005] + [0.001] * (len(dates) - 3), index=dates)
# write_csv(returns, 1)

# group_01 = (
#     pd.read_csv(
#         filepath_or_buffer='Group_01.csv',
#         parse_dates=['Date'],
#         na_values=[-99.99, -999]
#     )
#     .assign(Date=lambda x: x['Date'].dt.to_period('M'))
#     .set_index('Date')
#     .rename_axis(columns=['Team'])
#     .div(100)
# )
write_csv(momentumData['Momentum Returns'], group_number=24)